In [48]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


In [41]:
import urllib3

In [42]:
http = urllib3.PoolManager()

In [43]:
from config_batch import * 

# Functions

In [44]:
ws_hostname = "127.0.1.1"
ws_hostname = "10.1.0.45"

# ws_hostname = "192.168.1.3"


In [45]:
def call_ws(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search/?{params}"
    print(url)
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             print(res)
            res["time"] = datetime.now() - t
            return res
    except Exception as e:
        return str(e)
    

In [16]:
def call_ws_batch(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
#     print(addr_data)
#     print(addr_data.shape)
#     print()
    file_data = addr_data.rename(columns = {
        street_field : "street",
        housenbr_field: "housenumber",
        postcode_field: "postcode",
        city_field: "city",
        country_field: "country",
        addr_key_field : "addr_key"
    }).to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "with_rejected" : "yes" if with_reject else "no",
        "check_result" : "yes" if check_result else "no",
        "struct_osm" : "yes" if structured_osm else "no"
    })
    
    try:
        res = pd.DataFrame(json.loads(r.data.decode('utf-8')))
    except ValueError:
        print("Cannot decode result:")
        print(json.loads(r.data.decode('utf-8')))
        return 
#     display(res)
    return res

In [46]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d: "error" if "error" in d else "match" if "match" in d else "rejected")
    addresses["time"]  = addresses.json.apply(lambda d: d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: d["timing"] if "timing" in d else {})

    addresses["method"]= addresses.json.apply(lambda d: d["match"][0]["method"] if len(d)>0 and "match" in d else "none")
    
    for field in ["street", "number", "postcode", "city"]:
        addresses[field]= addresses.json.apply(lambda d: d["match"][0]["addr_out_"+field] if len(d)>0 and "match" in d else "")
    return 

# Calls

## Single address calls

In [47]:
call_ws({street_field:   "Av. Fonsny", 
         housenbr_field: "20",
         city_field:     "Saint-Gilles",
         postcode_field: "1060",
         country_field:  "Belgium"}, check_result=True, structured_osm=False)

http://10.1.0.45:5000/search/?street=Av.+Fonsny&housenumber=20&city=Saint-Gilles&postcode=1060&country=Belgium&check_result=yes&struct_osm=no


{'match': [{'SIM_city': 0.46153846153846156,
   'SIM_house_nbr': 1.0,
   'SIM_street': 1.0,
   'SIM_street_which': 'addr_out_street',
   'SIM_zip': 1.0,
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': 'DAE (Smals), 20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
   'extra_house_nbr': '20',
   'lat': '50.8358216',
   'lon': '4.3386884',
   'method': 'orig',
   'osm_addr_in': 'Av. Fonsny, 20, 1060 Saint-Gilles, Belgium',
   'place_id': 320313,
   'place_rank': 30}],
 'rejected': [{'SIM_city': 0.46153846153846156,
   'SIM_house_nbr': 1.0,
   'SIM_street': 1.0,
   'SIM_street_which': 'addr_out_street',
   'SIM_zip': 1.0,
   'addr_out_city': 'Saint-Gilles - S

In [21]:
call_ws({street_field:   "", 
         housenbr_field: "",
         city_field:     "Dinant",
         postcode_field: "5500",
         country_field:  "Belgium"}, check_result=True, structured_osm=True)

http://10.1.0.45:5000/search/?street=&housenumber=&city=Dinant&postcode=5500&country=Belgium&check_result=yes&struct_osm=yes


{'match': [{'SIM_city': 1.0,
   'SIM_house_nbr': 0.0,
   'SIM_street': 1.0,
   'SIM_street_which': 'addr_out_street',
   'SIM_zip': 0.1,
   'addr_out_city': 'Dinant',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '',
   'addr_out_postcode': '',
   'addr_out_street': '',
   'display_name': 'Dinant, Namur, Wallonie, België / Belgique / Belgien',
   'extra_house_nbr': '',
   'lat': '50.2591576',
   'lon': '4.9130624',
   'method': 'orig',
   'osm_addr_in': '5500 Dinant, Belgium',
   'place_id': 4067001,
   'place_rank': 14}],
 'rejected': [],
 'time': datetime.timedelta(microseconds=276415)}

In [11]:
call_ws({street_field:   "Fechtergasse", 
         housenbr_field: "16/13",
         city_field:     "Wenen",
         postcode_field: "1090",
         country_field:  "Oostenrijk"}, check_result=False, structured_osm=False)

http://10.1.0.45:5000/search/?street=Fechtergasse&housenumber=16%2F13&city=Wenen&postcode=1090&country=Oostenrijk&check_result=no&struct_osm=no


{'match': [{'SIM_street_which': '',
   'addr_out_city': 'Alsergrund',
   'addr_out_country': 'Österreich',
   'addr_out_number': '',
   'addr_out_postcode': '1090',
   'addr_out_street': '',
   'display_name': 'KG Alsergrund, Alsergrund, Wien, 1090, Österreich',
   'extra_house_nbr': '16/13',
   'lat': '48.22281431273457',
   'lon': '16.3564883123584',
   'method': 'nostreet',
   'osm_addr_in': '1090 Wenen, Oostenrijk',
   'place_id': 504963,
   'place_rank': 21}],
 'rejected': [],
 'time': datetime.timedelta(microseconds=680894)}

In [12]:
call_ws({street_field:   "Fechtergasse 16/13 1090 Wenen", 
         housenbr_field: "",
         city_field:     "",
         postcode_field: "",
         country_field:  "Oostenrijk"}, check_result=False, structured_osm=False)

http://10.1.0.45:5000/search/?street=Fechtergasse+16%2F13+1090+Wenen&housenumber=&city=&postcode=&country=Oostenrijk&check_result=no&struct_osm=no


{'match': [{'SIM_street_which': '',
   'addr_out_city': '',
   'addr_out_country': 'Österreich',
   'addr_out_number': '',
   'addr_out_postcode': '',
   'addr_out_street': '',
   'display_name': 'Österreich',
   'extra_house_nbr': '16/13',
   'lat': '47.2',
   'lon': '13.2',
   'method': 'nostreet',
   'osm_addr_in': 'Oostenrijk',
   'place_id': 470737,
   'place_rank': 4}],
 'rejected': [],
 'time': datetime.timedelta(microseconds=534976)}

## Batch calls (row by row)

In [38]:
addresses = get_addresses("address.csv.gz")
addresses = addresses.sample(100).copy()

### Simple way

In [74]:
addresses["json"] = addresses.progress_apply(call_ws, check_result=True, structured_osm=False, axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

### Using Dask

In [17]:
dd_addresses = dd.from_pandas(addresses, npartitions=4)

dask_task = dd_addresses.apply(call_ws, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses["json"] = dask_task.compute()

[########################################] | 100% Completed | 59.1s


In [26]:
expand_json(addresses)

In [27]:
addresses

,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber,json,status,time,timing,method,street,number,postcode,city
1321156,0836.380.520,Belgique,7911,Frasnes-lez-Anvaing,Rue Goderneau(MB),14,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.960761,{},regex[init],Rue Goderneau,,7906,Ath
2660061,2.231.595.460,Belgique,9981,Sint-Laureins,Vlamingstraat(STM),92,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:01.143796,{},regex[init],Vlamingstraat,,9981,Sint-Laureins
164680,0431.356.723,Belgique,1050,Ixelles,Rue de l'Amazone,21,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.724923,{},orig,Rue de l'Amazone - Amazonestraat,21,1050,Ixelles - Elsene
1295595,0832.380.556,Belgique,4000,Liège,Rue Henri-Maus,29,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.584219,{},orig,Rue Henri Maus,,4000,Liège
2293307,2.168.200.715,Belgique,8300,Knokke-Heist,Zeedijk-Knokke,641,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.548851,{},orig,Zeedijk Knokke,,8300,Knokke-Heist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751112,2.244.614.642,Belgique,9240,Zele,Smeiersberg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.544249,{},orig,Smeiersberg,,9240,Zele
2810314,2.252.606.947,Belgique,1080,Molenbeek-Saint-Jean,Rue Henri De Saegher,27,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.968186,{},orig,Rue Henri De Saegher - Henri De Saegherstraat,27,1080,Molenbeek-Saint-Jean - Sint-Jans-Molenbeek
732484,0650.498.331,Belgique,4700,Eupen,Stockbergerweg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.987093,{},orig,Stockbergerweg,,4700,Eupen
1799145,2.037.557.947,Belgique,1180,Uccle,Parvis Saint-Pierre,119,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.517703,{},orig,Parvis Saint-Pierre - Sint-Pietersvoorplein,,1180,Uccle - Ukkel


## Batch calls (batch WS)

### Single block

In [39]:
# Only geocoding
# addresses["StreetFR"] = ""
call_ws_batch(addresses, mode="geo", check_result=True, structured_osm=True)

,addr_key,lat,lon,place_rank,method
0,0413.290.175,50.628481,5.560296,30.0,orig
1,0417.333.788,50.918041,3.955867,30.0,orig
2,0434.930.281,51.112428,4.364174,30.0,orig
3,0438.277.672,50.464710,4.874722,30.0,orig
4,0458.703.496,50.725594,4.504176,26.0,orig
...,...,...,...,...,...
94,0819.219.537,50.627072,5.617789,30.0,libpostal+regex[lpost]+photon
95,2.077.530.756,50.689293,4.666302,26.0,libpostal+regex[lpost]+photon
96,2.254.201.410,50.769134,4.602587,30.0,libpostal+regex[lpost]+photon
97,2.285.876.264,50.163328,4.962765,30.0,libpostal+regex[lpost]+photon


In [62]:
# Geocode + address
call_ws_batch(addresses, mode="short") 

,addr_key,lat,lon,place_rank,method,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country
0,2.164.397.226,50.712641,4.530106,26.0,orig,Rue Robert Boisacq,None,9 A,1330,Rixensart,België / Belgique / Belgien
1,2.279.599.374,50.859702,2.828440,26.0,orig,Stijn Streuvelswijk,None,18,8908,Ieper,België / Belgique / Belgien
2,0669.833.203,51.209175,4.470072,30.0,orig,Boterlaarbaan,239,239,2100,Deurne,België / Belgique / Belgien
3,2.243.108.073,50.658359,5.483832,26.0,orig,Rue de l'Aéropostale,None,8,4460,Grâce-Hollogne,België / Belgique / Belgien
4,2.045.477.107,51.100407,3.167726,26.0,orig,Koningin Astridstraat,None,109,8210,Zedelgem,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...
92,0857.952.627,50.834163,3.231044,26.0,regex[init],Heulsestraat,None,3,8501,Kortrijk,België / Belgique / Belgien
93,2.249.583.121,51.194730,4.407469,30.0,orig,Desguinlei,88-90,90,2018,Antwerpen,België / Belgique / Belgien
94,0834.315.707,51.204519,4.398727,30.0,orig,Diercxsensstraat,39,39,2018,Antwerpen,België / Belgique / Belgien
95,0857.635.002,50.843955,4.264415,26.0,orig,Ninoofsesteenweg,None,227,1700,Dilbeek,België / Belgique / Belgien


In [63]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_reject=True) 

,addr_key,country,postcode,city,street,housenumber,osm_addr_in,place_id,lat,lon,...,addr_out_street,addr_out_city,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,retry_on_26,method,extra_house_nbr,rejected
0,2.164.397.226,Belgique,1330,Rixensart,Rue Robert Boisacq,9 A,"Rue Robert Boisacq, 9 A, 1330 Rixensart, Belgique",564275.0,50.712641,4.530106,...,Rue Robert Boisacq,Rixensart,None,België / Belgique / Belgien,1330,Fond du Patch,None,orig,9 A,None
1,2.279.599.374,Belgique,8908,Ieper,Streuvelswijk,18,"Streuvelswijk, 18, 8908 Ieper, Belgique",785053.0,50.859702,2.828440,...,Stijn Streuvelswijk,Ieper,None,België / Belgique / Belgien,8908,,None,orig,18,None
2,0669.833.203,Belgique,2100,Antwerpen,Boterlaarbaan,239,"Boterlaarbaan, 239, 2100 Antwerpen, Belgique",2010239.0,51.209175,4.470072,...,Boterlaarbaan,Deurne,239,België / Belgique / Belgien,2100,Eksterlaar,None,orig,239,None
3,2.243.108.073,Belgique,4460,Grâce-Hollogne,Rue de l'Aéropostale,8,"Rue de l'Aéropostale, 8, 4460 Grâce-Hollogne, ...",1445516.0,50.658359,5.483832,...,Rue de l'Aéropostale,Grâce-Hollogne,None,België / Belgique / Belgien,4460,Liège logistics,None,orig,8,None
4,2.045.477.107,Belgique,8210,Zedelgem,Koningin Astridstraat,109,"Koningin Astridstraat, 109, 8210 Zedelgem, Bel...",1763661.0,51.100407,3.167726,...,Koningin Astridstraat,Zedelgem,None,België / Belgique / Belgien,8210,,None,orig,109,"[{'index': 5.0, 'osm_addr_in': 'Koningin Astri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0857.952.627,Belgique,8501,Kortrijk,Heulsestraat(Bis),3,"Heulsestraat, 3, 8501 Kortrijk, Belgique",2738948.0,50.834163,3.231044,...,Heulsestraat,Kortrijk,None,België / Belgique / Belgien,8501,,None,regex[init],3,None
93,2.249.583.121,Belgique,2018,Antwerpen,Desguinlei,90,"Desguinlei, 90, 2018 Antwerpen, Belgique",2323301.0,51.194730,4.407469,...,Desguinlei,Antwerpen,88-90,België / Belgique / Belgien,2018,Markgrave,None,orig,90,None
94,0834.315.707,Belgique,2018,Antwerpen,Diercxsensstraat,39,"Diercxsensstraat, 39, 2018 Antwerpen, Belgique",2338350.0,51.204519,4.398727,...,Diercxsensstraat,Antwerpen,39,België / Belgique / Belgien,2018,Brederode,None,orig,39,None
95,0857.635.002,Belgique,1700,Dilbeek,Ninoofsesteenweg,227,"Ninoofsesteenweg, 227, 1700 Dilbeek, Belgique",943444.0,50.843955,4.264415,...,Ninoofsesteenweg,Dilbeek,None,België / Belgique / Belgien,1700,,None,orig,227,"[{'index': 120.0, 'osm_addr_in': 'Ninoofsestee..."


### Batch blocs

In [21]:
def call_ws_batch_chunks(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False, chunk_size=100): 
    ## TODO : find a better way with dask? It seems that map_partitions does not support function returning dataframes. 

    chunks = np.array_split(addr_data, addr_data.shape[0]//chunk_size)

    res= [call_ws_batch(chunk, mode=mode, 
                        check_result=check_result, 
                        structured_osm=structured_osm) for chunk in tqdm(chunks)]
    df_res = pd.concat(res, sort=False)
    return df_res

In [ ]:
df_res = call_ws_batch_chunks(addresses, chunk_size=10)
df_res

In [1]:
df_res.method.value_counts()

NameError: name 'df_res' is not defined

## Comparing options

In [19]:
addresses = get_addresses("address.csv.gz")
addresses = addresses[addresses[country_field] == "Belgique"]
addresses = addresses.sample(10000).copy()

In [22]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch_chunks(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

check struct


  0%|          | 0/100 [00:00<?, ?it/s]

check unstruct


  0%|          | 0/100 [00:00<?, ?it/s]

nocheck struct


  0%|          | 0/100 [00:00<?, ?it/s]

nocheck unstruct


  0%|          | 0/100 [00:00<?, ?it/s]

Iterations per seconds:


,struct,unstruct
check,2.725135,11.452610
nocheck,21.512587,16.940754


In [23]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

Match rate


,check,nocheck
struct,0.9990,1.0
unstruct,0.9999,1.0


In [24]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)].query("method!='nostreet'").shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

Match rate (without nostreet)


,check,nocheck
struct,0.9875,1.0000
unstruct,0.9878,0.9999


In [25]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1]["addr_key"])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

Unmatched addresses
('check', 'struct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
1090480,0722.823.214,Belgique,9220,Hamme (Vl.),Kapellestraat,5
678318,0637.726.696,Belgique,9220,Hamme (Vl.),Boteraard,1
1382553,0845.909.482,Belgique,5560,Houyet,"Rue d'Inzepia,M.-St-Blaise",22
1487535,0867.111.605,Belgique,9185,Wachtebeke,Plassenstraat,6
2996396,2.275.184.290,Belgique,8551,Zwevegem,Remi Deprezstraat(H),1
286262,0454.059.374,Belgique,7110,La Louvière,Rue Dr. Coffé(S-B),106
2979811,2.273.336.540,Belgique,4257,Berloz,Chaussée de Nivelles,14
2785901,2.249.394.465,Belgique,8460,Oudenburg,Waerevaartstraat,1
867076,0677.701.485,Belgique,5537,Anhée,Grange,3
1410228,0850.220.836,Belgique,6941,Ganshoren,PARC RESIDENTIEL,NaN


Belgique    10
Name: CountryFR, dtype: int64
('check', 'unstruct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
1410228,0850.220.836,Belgique,6941,Ganshoren,PARC RESIDENTIEL,NaN


Belgique    1
Name: CountryFR, dtype: int64
('nocheck', 'struct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber


Series([], Name: CountryFR, dtype: int64)
('nocheck', 'unstruct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber


Series([], Name: CountryFR, dtype: int64)


In [26]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())

for k1 in results:
    vc_values.loc[k1, k1] = results[k1].shape[0]
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
 
            vc = mg._merge.value_counts()

            mismatches = mg[mg.place_id_x != mg.place_id_y][["addr_key"]]
            mismatches = mismatches.merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))
            mismatches = mismatches.merge(r1[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches = mismatches.merge(r2[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*5 + [f"y:{k2}"]*5, mismatches.columns])

            mismatch_values = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).fillna("") != 
                                          mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).fillna("")).any(axis=1)]
            
            mismatch_values_no_nmbr = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).drop("addr_out_number", axis=1).fillna("") != 
                                                  mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).drop("addr_out_number", axis=1).fillna("")).any(axis=1)]
            
            
            vc_label = f"{vc['both']} ({mismatches.shape[0]} - {mismatch_values.shape[0]} - {mismatch_values_no_nmbr.shape[0]}) / {vc['left_only']} / {vc['right_only']}"
            vc_values.loc[k1, k2]=vc_label

                
            print(f"{k1} vs {k2}")
            print(vc_label)
            print("-----------------------------")
            
            print(f"Only in {k1}")
            display(r1[r1.addr_key.isin(mg[mg._merge=="left_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print(f"Only in {k2}")
            display(r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print("Mismatch on place_id")
            display(mismatches)
            
            print("Mismatch on values")
            
            display(mismatch_values)
            
            print("Mismatch on values (no nbr)")
            display(mismatch_values_no_nmbr)
            
            print("#######################")
            
# display(vc_values.fillna(""))

('check', 'unstruct') vs ('check', 'struct')
9990 (352 - 304 - 226) / 9 / 0
-----------------------------
Only in ('check', 'unstruct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0722.823.214,51.097388,4.139718,30.0,libpostal+regex[lpost]+photon,2667276.0,Kapellestraat,5,5,9220,Hamme,België / Belgique / Belgien,Belgique,9220,Hamme (Vl.),Kapellestraat,5
1,0637.726.696,51.075010,4.108771,26.0,libpostal+regex[lpost]+photon,656842.0,Boteraard,None,1,9220,Hamme,België / Belgique / Belgien,Belgique,9220,Hamme (Vl.),Boteraard,1
2,0845.909.482,50.168734,4.890450,30.0,orig,2930668.0,Rue d'Inzepia,22,22,5560,Mesnil-Saint-Blaise,België / Belgique / Belgien,Belgique,5560,Houyet,"Rue d'Inzepia,M.-St-Blaise",22
3,0867.111.605,51.193476,3.862197,21.0,nostreet,4068706.0,None,None,6,9185,Wachtebeke,België / Belgique / Belgien,Belgique,9185,Wachtebeke,Plassenstraat,6
4,2.275.184.290,50.805224,3.383866,21.0,nostreet,4068607.0,None,None,1,8551,Heestert,België / Belgique / Belgien,Belgique,8551,Zwevegem,Remi Deprezstraat(H),1
5,0454.059.374,50.482393,4.144590,21.0,nostreet,4068442.0,None,None,106,7110,La Louvière,België / Belgique / Belgien,Belgique,7110,La Louvière,Rue Dr. Coffé(S-B),106
6,2.273.336.540,50.697946,5.210160,21.0,nostreet,4068023.0,None,None,14,4257,Berloz,België / Belgique / Belgien,Belgique,4257,Berloz,Chaussée de Nivelles,14
7,2.249.394.465,51.158949,2.997841,22.0,nonum,711510.0,De Waerebrug,None,1,8460,Gistel,België / Belgique / Belgien,Belgique,8460,Oudenburg,Waerevaartstraat,1
8,0677.701.485,50.294604,4.870586,20.0,orig,448877.0,Grange,None,3,5523,Anhée,België / Belgique / Belgien,Belgique,5537,Anhée,Grange,3


Only in ('check', 'struct')


,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,addr_key,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber


Mismatch on place_id


...

Mismatch on values


Input                                      \
          addr_key CountryFR Zipcode    MunicipalityFR   
0     0504.892.522  Belgique    6640     Vaux-sur-Sûre   
1     0714.809.430  Belgique    6660        Houffalize   
2    2.237.623.417  Belgique    6997            Erezée   
3    2.268.170.992  Belgique    1495  Villers-la-Ville   
5    2.217.934.197  Belgique    4960           Malmedy   
..             ...       ...     ...               ...   
719  2.126.385.104  Belgique    5555            Bièvre   
721   0806.826.796  Belgique    5310           Eghezée   
722  2.011.526.216  Belgique    7972           Beloeil   
731   0691.715.908  Belgique    7141        Morlanwelz   
732  2.067.726.333  Belgique    4780        Sankt Vith   

                                               x:('nocheck', 'unstruct')  \
                          StreetFR HouseNumber         addr_out_street_x   
0      Chaussée de Bastogne,Sibret          88      Chaussée de Bastogne   
1    R. de la Villa-Romaine,Nadrin          35   Rue de la Villa Romaine   
2       Rue Vivier-del-Haie, Biron           2       Rue Vivier-del-Haie   
3         Rue de Priesmont,Marbais         173          Rue de Priesmont   
5          Mont-Chemin de Rîbômont          13                      None   
..                             ...         ...                       ...   
719       Vôye du Sclassin, Graide          47          Vôye du Sclassin   
721         Rue des Acacias,Aische           1           Rue des Acacias   
722       Chaussée Brunehaut (Que)         106     Chaussée de Brunehaut   
731        Rue Vieille Eglise(CAR)          36                      None   
732                 Südstraße,Rodt         120                 Südstraße   

                                                             \
    addr_out_number_x extra_house_nbr_x addr_out_postcode_x   
0                  88                88                6640   
1                  35                35                6982   
2                   2                 2                6997   
3                None               173                1495   
5                None                13                None   
..                ...               ...                 ...   
719                47                47                5555   
721                 1                 1                5310   
722              None               106                7972   
731              None                36                None   
732              None               120                4780   

                             y:('nocheck', 'struct')                    \
           addr_out_city_x         addr_out_street_y addr_out_number_y   
0                   Sibret      Chaussée de Bastogne              None   
1                   Nadrin   Rue de la Villa Romaine              None   
2                    Biron                      None              None   
3         Villers-la-Ville                      None              None   
5                     None        Chemin de Rîbômont                13   
..                     ...                       ...               ...   
719                 Graide                      None              None   
721       Aische-En-Refail                      None              None   
722  Ellignies-Sainte-Anne     Chaussée de Brunehaut              None   
731                   None  Rue de la Vieille Eglise              None   
732             Sankt Vith                      None              None   

                                                           
    extra_house_nbr_y addr_out_postcode_y addr_out_city_y  
0                  88                6640       Villeroux  
1                  35                6982          Nadrin  
2                   2                None            None  
3                 173                None            None  
5                  13                4960         Malmedy  
..                ...                 ...             ...  
719   

Mismatch on values (no nbr)


Input                                      \
          addr_key CountryFR Zipcode    MunicipalityFR   
0     0504.892.522  Belgique    6640     Vaux-sur-Sûre   
2    2.237.623.417  Belgique    6997            Erezée   
3    2.268.170.992  Belgique    1495  Villers-la-Ville   
5    2.217.934.197  Belgique    4960           Malmedy   
10   2.011.992.806  Belgique    6790           Aubange   
..             ...       ...     ...               ...   
719  2.126.385.104  Belgique    5555            Bièvre   
721   0806.826.796  Belgique    5310           Eghezée   
722  2.011.526.216  Belgique    7972           Beloeil   
731   0691.715.908  Belgique    7141        Morlanwelz   
732  2.067.726.333  Belgique    4780        Sankt Vith   

                                             x:('nocheck', 'unstruct')  \
                        StreetFR HouseNumber         addr_out_street_x   
0    Chaussée de Bastogne,Sibret          88      Chaussée de Bastogne   
2     Rue Vivier-del-Haie, Biron           2       Rue Vivier-del-Haie   
3       Rue de Priesmont,Marbais         173          Rue de Priesmont   
5        Mont-Chemin de Rîbômont          13                      None   
10   Complexe douanier - AUBANGE          SN         Complexe douanier   
..                           ...         ...                       ...   
719     Vôye du Sclassin, Graide          47          Vôye du Sclassin   
721       Rue des Acacias,Aische           1           Rue des Acacias   
722     Chaussée Brunehaut (Que)         106     Chaussée de Brunehaut   
731      Rue Vieille Eglise(CAR)          36                      None   
732               Südstraße,Rodt         120                 Südstraße   

                                                             \
    addr_out_number_x extra_house_nbr_x addr_out_postcode_x   
0                  88                88                6640   
2                   2                 2                6997   
3                None               173                1495   
5                None                13                None   
10               None                SN                6790   
..                ...               ...                 ...   
719                47                47                5555   
721                 1                 1                5310   
722              None               106                7972   
731              None                36                None   
732              None               120                4780   

                             y:('nocheck', 'struct')                    \
           addr_out_city_x         addr_out_street_y addr_out_number_y   
0                   Sibret      Chaussée de Bastogne              None   
2                    Biron                      None              None   
3         Villers-la-Ville                      None              None   
5                     None        Chemin de Rîbômont                13   
10                 Aubange                      None              None   
..                     ...                       ...               ...   
719                 Graide                      None              None   
721       Aische-En-Refail                      None              None   
722  Ellignies-Sainte-Anne     Chaussée de Brunehaut              None   
731                   None  Rue de la Vieille Eglise              None   
732             Sankt Vith                      None              None   

                                                           
    extra_house_nbr_y addr_out_postcode_y addr_out_city_y  
0                  88                6640       Villeroux  
2                   2                None            None  
3                 173                None            None  
5                  13                4960         Malmedy  
10                 SN                None            None  
..                ...                 ...             ...  
719                47           

#######################


In [27]:
print("Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns")
vc_values.fillna("")

Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns


check  \
                                           struct   
check   struct                               9990   
        unstruct   9990 (352 - 304 - 226) / 9 / 0   
nocheck struct    9990 (725 - 724 - 717) / 10 / 0   
        unstruct  9990 (782 - 741 - 662) / 10 / 0   

                                                  \
                                        unstruct   
check   struct                                     
        unstruct                            9999   
nocheck struct    9999 (899 - 858 - 782) / 1 / 0   
        unstruct  9999 (592 - 591 - 583) / 1 / 0   

                                          nocheck           
                                           struct unstruct  
check   struct                                              
        unstruct                                            
nocheck struct                              10000           
        unstruct  10000 (735 - 413 - 349) / 0 / 0    10000